In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from store import *
import kbech32

In [2]:
store = Store(os.getenv('localappdata') + r'\Kaspad\kaspa-mainnet\datadir2')

In [3]:
store.load_recent_blocks(7200_000)

In [4]:
header_fields = ['timeInMilliseconds', 'blueScore', 'blueWork', 'daaScore', 'difficulty']
block_fields = ['pubkey_script', 'kaspad_version', 'miner_version']
count_fields = [] # ['num_parents', 'num_children', 'num_blues', 'num_reds']
frames = store.load_data(header_fields=header_fields, block_fields=block_fields, count_fields=count_fields)

  0%|          | 0/7230 [00:00<?, ?it/s]

In [5]:
frames.keys()

dict_keys(['hash', 'timeInMilliseconds', 'blueScore', 'blueWork', 'daaScore', 'difficulty', 'pubkey_script', 'kaspad_version', 'miner_version'])

In [6]:
df = pd.DataFrame(frames).set_index('hash')

In [7]:
pp_time = store.get_header_data(store.pruning_point()).timeInMilliseconds
pp_time, len(df)

(1663284609692, 7230)

In [8]:
df['kaspad_version'].value_counts(normalize=True)

0.12.6    0.904564
0.12.4    0.069986
0.12.5    0.025450
Name: kaspad_version, dtype: float64

In [24]:
def miner_type(miner_version):
    if 'BzMiner' in miner_version:
        return 'BzMiner'
    if '0.2.1-GPU' in miner_version:
        return 'CommunityMiner'
#     if 'WoolyPooly' in miner_version:
#         return 'WoolyPooly'
#     if 'acc-pool' in miner_version:
#         return 'acc-pool'
    if 'KStratum' in miner_version:
        return 'KStratum'
    if 'onemorebsmith' in miner_version:
        return 'onemorebsmith-bridge'
    return miner_version

df["miner_type"] = df["miner_version"].apply(miner_type)

In [25]:
(df['kaspad_version'] + '/' + df['miner_type']).value_counts(normalize=True)

0.12.6/WoolyPooly DE4          0.122407
0.12.6/kaspaminer-0.12.6       0.086307
0.12.6/WoolyPooly SG1          0.085615
0.12.6/eu1.acc-pool.pw         0.081743
0.12.6/WoolyPooly DE3          0.075380
0.12.6/WoolyPooly TR1          0.071231
0.12.4/00                      0.069571
0.12.6/WoolyPooly RU1          0.057815
0.12.6/WoolyPooly KR1          0.056570
0.12.6/WoolyPooly BR1          0.047580
0.12.6/WoolyPooly US1          0.045505
0.12.6/kaspaminer-0.12.2       0.036238
0.12.6/ru.acc-pool.pw          0.032503
0.12.6/hk.acc-pool.pw          0.025864
0.12.6/ca.acc-pool.pw          0.025311
0.12.5/CommunityMiner          0.021162
0.12.6/WoolyPooly CA1          0.013140
0.12.6/KStratum                0.012310
0.12.6/kaspa-pool.org          0.010512
0.12.6/BzMiner                 0.007331
0.12.6/WoolyPooly AU1          0.004979
0.12.6/ge.acc-pool.pw          0.003873
0.12.5/KStratum                0.002213
0.12.5/eu2.acc-pool.pw         0.001798
0.12.6/CommunityMiner          0.000830


In [26]:
df['miner_type'].value_counts(normalize=True)

WoolyPooly DE4          0.122407
kaspaminer-0.12.6       0.086307
WoolyPooly SG1          0.085615
eu1.acc-pool.pw         0.081743
WoolyPooly DE3          0.075380
WoolyPooly TR1          0.071231
00                      0.069571
WoolyPooly RU1          0.057815
WoolyPooly KR1          0.056570
WoolyPooly BR1          0.047580
WoolyPooly US1          0.045505
kaspaminer-0.12.2       0.036238
ru.acc-pool.pw          0.032503
hk.acc-pool.pw          0.025864
ca.acc-pool.pw          0.025311
CommunityMiner          0.021992
KStratum                0.014799
WoolyPooly CA1          0.013140
kaspa-pool.org          0.010512
BzMiner                 0.007607
WoolyPooly AU1          0.004979
ge.acc-pool.pw          0.003873
eu2.acc-pool.pw         0.001798
ru.solo.acc-pool.pw     0.000830
ge.solo.acc-pool.pw     0.000415
ca.solo.acc-pool.pw     0.000277
onemorebsmith-bridge    0.000138
Name: miner_type, dtype: float64

In [12]:
store.close()